**Design principle**

- Ultrasensitive negative feedback can generate periodic oscillations in cells.
- Oscillator phase can be used as a growth timer.

**Techniques**

- Composition of functions
- Linear stability analysis
- Linear stability diagrams
- Numerical calculation of a scalar fixed point
- Synthetic biology

**References**

- [Elowitz & Leibler, A synthetic oscillatory network of transcriptional regulators, *Nature*, 2000](https://doi.org/10.1038/35002125)
- [Synchronous long-term oscillations in a synthetic gene circuit, *Nature*, 2016](https://doi.org/10.1038/nature19841) 

<hr>

In [1]:
from pyquibbler import iquib, override_all, q, quiby
override_all()
import matplotlib.pyplot as plt
import numpy as np
%matplotlib tk

In [2]:
import scipy.integrate
import scipy.optimize

In [3]:
from matplotlib.widgets import Slider

In [4]:
beta_slider = iquib(1.)
gamma_slider = iquib(0.)
rho_slider = iquib(-3.)
n_slider = iquib(3.)

def repressilator_rhs(mx, t, beta, gamma, rho, n):
    """
    Returns 6-array of (dm_1/dt, dm_2/dt, dm_3/dt, dx_1/dt, dx_2/dt, dx_3/dt)
    """
    m_1, m_2, m_3, x_1, x_2, x_3 = mx
    return np.array(
        [
            beta * (rho + 1 / (1 + x_3 ** n)) - m_1,
            beta * (rho + 1 / (1 + x_1 ** n)) - m_2,
            beta * (rho + 1 / (1 + x_2 ** n)) - m_3,
            gamma * (m_1 - x_1),
            gamma * (m_2 - x_2),
            gamma * (m_3 - x_3),
        ]
    )


# Initial condiations
x0 = iquib(np.array([0, 0, 0, 1., 1.5, 2.2]))

# Number of points to use in plots
n_points = 1000


# Solve for species concentrations
@quiby
def _solve_repressilator(log_beta, log_gamma, log_rho, n, t_max, x_init):
    beta = 10 ** log_beta
    gamma = 10 ** log_gamma
    rho = 10 ** log_rho
    t = np.linspace(0, t_max, n_points)
    x = scipy.integrate.odeint(repressilator_rhs, x_init, t, args=(beta, gamma, rho, n))
    m1, m2, m3, x1, x2, x3 = x.transpose()
    return t, m1, m2, m3, x1, x2, x3


t, m1, m2, m3, x1, x2, x3 = _solve_repressilator(
    beta_slider,
    gamma_slider,
    rho_slider,
    n_slider,
    40.0,
    x0,
).iter_first()

plt.figure(1)
plt.clf()
plt.plot(t, x1, 'r', t, x2, 'g', t, x3, 'b');

plt.plot(0, x0[3], 'ro')
plt.plot(0, x0[4], 'go')
plt.plot(0, x0[5], 'bo');

In [7]:
plt.figure(figsize=(6, 2))
Slider(ax=plt.subplot(4, 1, 1), valmin=0,  valmax=3, valinit=beta_slider, label='beta')
Slider(ax=plt.subplot(4, 1, 2), valmin=0,  valmax=2, valinit=gamma_slider, label='gamma')
Slider(ax=plt.subplot(4, 1, 3), valmin=-5, valmax=0, valinit=rho_slider, label='rho')
Slider(ax=plt.subplot(4, 1, 4), valmin=0,  valmax=5, valinit=n_slider, label='n_slider');